In [1]:
# Maddy Peng -- 2/10/25

## Tobias workflow

Starting from filtering the BAM files with given barcodes for each sample and cell type.
**See R script for generating these barcodes and peak files**

In [1]:
# Install pysam if you don't have it already
!pip install pysam

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import subprocess
import pysam
import pandas as pd
import numpy as np
import os
import re

In [2]:
# Get working directory
!pwd

/groups/mb928_gp/mp4486/tobias


In [3]:
# Read in metadata to get diagnoses
meta = pd.read_csv("../Multiome_process/meta/current_working_multiome_meta (1).csv")

/tmp/mp4486/ipykernel_3274144/2772505042.py:2: DtypeWarning: Columns (28,31,32,38,46) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv("../Multiome_process/meta/current_working_multiome_meta (1).csv")


In [4]:
meta['sample']

0         CM021
1         CM021
2         CM021
3         CM021
4         CM021
          ...  
495032    MM152
495033    MM152
495034    MM152
495035    MM152
495036    MM152
Name: sample, Length: 495037, dtype: object

In [5]:
# Get IDs from the metadata
ids = list(set(meta['sample']))
len(ids)

74

In [7]:
cell_types = ['NSC.a','NSC.b', 'INP', 'NB','ImGC.1','ImGC.2','GC.1']

In [8]:
# Get all the bam file paths per sample
def get_bam_path(i):
    if i[:2] == 'CM':
        path = "/groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/"

    else:
        if re.match(r"^(00[1-9]|0[12][0-9]|03[01])$", i[2:5]):
            path = "/groups/mb928_gp/data_backup/230817_MAURA_MADELINE_12_HUMAN_10X/bamfiles/"

        elif re.match(r"^(14[5-9]|15[0-2])$", i[2:5]):
            path = "/groups/mb928_gp/data_backup/250131_MAURA_MADELINE_7_HUMAN_LIBRARY_SELF_3B_PE150_X/BAM_Multiome/"
        else:    
            path = "/groups/mb928_gp/data_backup/240806_MAURA_MADELINE_6_HUMAN_LIBRARY_10X/BAM/"

    return path + f"{i}_atac_possorted_bam.bam"

    
bam_paths = [get_bam_path(i) for i in ids]

In [9]:
len(bam_paths)

74

In [9]:
# Define filtering function: Use pysam for filtering bam by barcode
def filter_bam_by_barcode(input_bam, barcode_file, output_bam):
    """Filter a BAM file to keep only reads with barcodes from a given list."""
    if os.path.exists(output_bam):
        print(f"{output_bam} already exits!")
        return

    if not os.path.exists(barcode_file):
        print(f"{barcode_file} does not exist ..... skipping .......")
        return

    print(input_bam)
        
    # Read barcode list from file
    with open(barcode_file, 'r') as f:
        barcodes = set(line.strip() for line in f)
    
    # Open input BAM and create an output BAM
    with pysam.AlignmentFile(input_bam, "r") as bam_in, \
         pysam.AlignmentFile(output_bam, "wb", template=bam_in) as bam_out:
        
        for read in bam_in:
            # Extract barcode from read (Assumes barcode is stored in a tag, e.g., 'CB' for 10x data)
            barcode = read.get_tag('CB') if read.has_tag('CB') else None
            
            if barcode and f"CB:Z:{barcode}" in barcodes:
                bam_out.write(read)

    print(f"Filtered BAM saved: {output_bam}")


def process_sample(args):
    c, sample_id, input_bam = args
    output_path = f"neurogenesis/filtered_bams/{c}"
    os.makedirs(output_path, exist_ok=True)

    barcode_file = f"neurogenesis/barcodes/{sample_id}_{c}.txt"
    output_bam = f"{output_path}/{sample_id}_filtered.bam"
    
    filter_bam_by_barcode(input_bam, barcode_file, output_bam)

In [10]:
# Check number of cores available
!nproc

128


In [11]:
from concurrent.futures import ProcessPoolExecutor


In [12]:
# Nested for loop: for each cell type loop through each sample 
from concurrent.futures import ProcessPoolExecutor

# Prepare all tasks
tasks = []
for c in cell_types:
    for i in range(len(ids)):
        tasks.append((c, ids[i], bam_paths[i]))

# Run in parallel
with ProcessPoolExecutor(max_workers=128) as executor:
    executor.map(process_sample, tasks)

/groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM088_atac_possorted_bam.bam/groups/mb928_gp/data_backup/240806_MAURA_MADELINE_6_HUMAN_LIBRARY_10X/BAM/MM081_atac_possorted_bam.bam/groups/mb928_gp/data_backup/230817_MAURA_MADELINE_12_HUMAN_10X/bamfiles/MM008_atac_possorted_bam.bam/groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM027_atac_possorted_bam.bam/groups/mb928_gp/data_backup/230817_MAURA_MADELINE_12_HUMAN_10X/bamfiles/MM016_atac_possorted_bam.bam/groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM089_atac_possorted_bam.bam/groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM109_atac_possorted_bam.bam/groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM074_atac_possorted_bam.bam/groups/mb928_gp/data_backup/230817_MAURA_MADELINE

[W::hts_idx_load3] The index file is older than the data file: /groups/mb928_gp/data_backup/230817_MAURA_MADELINE_12_HUMAN_10X/bamfiles/MM002_atac_possorted_bam.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM092_atac_possorted_bam.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM059_atac_possorted_bam.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM038_atac_possorted_bam.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /groups/mb928_gp/data_backup/ngs/release/singleCell/220502_MAURA_CHEICK_16_HUMAN_SEQUENCING/bamfiles/CM087_atac_possorted_bam.bam.bai
[W::hts_idx_load3] The index file is older 

Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM002_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/CM088_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/CM026_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/CM091_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM100_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM081_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM084_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/CM071_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM103_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/CM087_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM083_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/CM044_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM082_filtered.bam
Filtered BAM saved: neurogenesis/filtered_bams/GC.1/MM097_filtered.bam
Filter

In [13]:
# Define merge bam file function: use pysam to merge
def merge_bam_files(args):
    """Merge BAM files for a given cell type and condition."""
    cell_type, bam_list, output_dir = args
    
    if not bam_list:
        return f"No BAMs to merge for {cell_type}"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_path = os.path.join(output_dir, f"{cell_type}_merged.bam")

    if os.path.exists(output_path):
        print(f"{cell_type} already exists")
        return
    
    pysam.merge("-f", output_path, *bam_list)
    print(f"Merged BAM for {cell_type}: {output_path}")

# Prepare tasks
cell_types = ['GC.1']
merge_tasks = []
for c in cell_types:
    bam_path = f"neurogenesis/filtered_bams/{c}"

    bams = [f"{bam_path}/{i}_filtered.bam" for i in ids if os.path.exists(f"{bam_path}/{i}_filtered.bam")]

    merge_tasks.append((c, bams, f"neurogenesis/merged_bams/"))

In [14]:
merge_tasks

[('GC.1',
  ['neurogenesis/filtered_bams/GC.1/CM089_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/MM146_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM112_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/MM002_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/MM102_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM060_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM070_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/MM145_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM053_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM042_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/MM150_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM041_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM086_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM038_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/MM023_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/MM097_filtered.bam',
   'neurogenesis/filtered_bams/GC.1/CM072_filtered.bam',
   'neurogenesis/filt

In [15]:
# Run in parallel
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor(max_workers=8) as executor:
    executor.map(merge_bam_files, merge_tasks)

Merged BAM for GC.1: neurogenesis/merged_bams/GC.1_merged.bam


In [ ]:
!pip install tobias

In [14]:
cell_types

['NSC.a', 'NSC.b', 'INP', 'NB', 'ImGC.1', 'ImGC.2', 'GC.1']

In [16]:
# Run TOBIAS ATACorrect

import os
import subprocess
from concurrent.futures import ProcessPoolExecutor

# Constants
genome_fasta = "/groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/genome.fa"
blacklist = "/groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/ENCFF356LFX_blacklist.bed"
peaks_dir = 'neurogenesis/peaks'
output_root = 'neurogenesis/merged_bams'

# Build tasks: (group, cell_type, bam_file, peak_file, output_dir)
tasks = []
for c in cell_types:
    bam_file = f"{output_root}/{c}_merged.bam"
    peak_file = f"{peaks_dir}/trajectory_ALL.bed"
    output_dir = f"{output_root}/ALL_compare"
    output_prefix = f"{output_dir}/{c}_corrected"

    if os.path.exists(bam_file) and os.path.exists(peak_file):
        tasks.append((bam_file, peak_file, output_prefix, output_dir))

# Function to run TOBIAS ATACorrect
def run_tobias(task):
    bam_file, peaks_file, output_prefix, outdir = task
    cell_type = os.path.basename(bam_file).split('.')[0]
    print(f"Running TOBIAS ATACorrect for {cell_type}")

    command = [
        "TOBIAS", "ATACorrect",
        "--bam", bam_file,
        "--genome", genome_fasta,
        "--blacklist", blacklist,
        "--peaks", peaks_file,
        "--outdir", outdir,
        "--prefix", os.path.basename(output_prefix),
        "--cores", "4"
    ]
    
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
    if result.stderr:
        print(result.stderr.decode())

# Run in parallel
with ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(run_tobias, tasks)

Running TOBIAS ATACorrect for INP_mergedRunning TOBIAS ATACorrect for NSCRunning TOBIAS ATACorrect for NB_mergedRunning TOBIAS ATACorrect for NSC



# TOBIAS 0.17.1 ATACorrect (run started 2025-11-05 09:38:04.025260)
# Working directory: /groups/mb928_gp/mp4486/tobias
# Command line call: TOBIAS ATACorrect --bam neurogenesis/merged_bams/NSC.a_merged.bam --genome /groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/genome.fa --blacklist /groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/ENCFF356LFX_blacklist.bed --peaks neurogenesis/peaks/trajectory_ALL.bed --outdir neurogenesis/merged_bams/ALL_compare --prefix NSC.a_corrected --cores 4

# ----- Input parameters -----
# bam:	neurogenesis/merged_bams/NSC.a_merged.bam
# genome:	/groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/genome.fa
# peaks:	neurogenesis/peaks/trajectory_ALL.bed
# regions_in:	None
# regions_out:	None
# blacklist:	/groups/mb928_gp/adr2189/TOBIAS/

In [17]:
# Run TOBIAS footprint scores

from concurrent.futures import ProcessPoolExecutor

peaks_dir = 'neurogenesis/peaks'
genome_dir = 'neurogenesis/merged_bams/ALL_compare'

def prepare_args(cell_type):
    corrected_file = f"{genome_dir}/{cell_type}_corrected_corrected.bw"
    peaks_file = f"{peaks_dir}/trajectory_ALL.bed"
    output_dir = f"{genome_dir}/{cell_type}_footprints"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"{cell_type}_footprints.bw")
    return (cell_type, corrected_file, peaks_file, output_file)

def process_task(args):
    cell_type, corrected_file, peaks_file, output_file = args
    print(f"Running TOBIAS FootprintScores for {cell_type}")
    
    command = [
        "TOBIAS", "FootprintScores",
        "--signal", corrected_file,
        "--regions", peaks_file,
        "--output", output_file,
        "--cores", "8"
    ]

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    print(f"[{cell_type}] STDOUT:\n{result.stdout.decode()}")
    print(f"[{cell_type}] STDERR:\n{result.stderr.decode()}")

# Create task list
args_list = [prepare_args(ct) for ct in cell_types]

# Run in parallel
with ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(process_task, args_list)

Running TOBIAS FootprintScores for NBRunning TOBIAS FootprintScores for NSC.bRunning TOBIAS FootprintScores for NSC.aRunning TOBIAS FootprintScores for INP



[NB] STDOUT:
# TOBIAS 0.17.1 ScoreBigwig (run started 2025-11-05 13:05:44.777289)
# Working directory: /groups/mb928_gp/mp4486/tobias
# Command line call: TOBIAS FootprintScores --signal neurogenesis/merged_bams/ALL_compare/NB_corrected_corrected.bw --regions neurogenesis/peaks/trajectory_ALL.bed --output neurogenesis/merged_bams/ALL_compare/NB_footprints/NB_footprints.bw --cores 8

# ----- Input parameters -----
# signal:	neurogenesis/merged_bams/ALL_compare/NB_corrected_corrected.bw
# output:	neurogenesis/merged_bams/ALL_compare/NB_footprints/NB_footprints.bw
# regions:	neurogenesis/peaks/trajectory_ALL.bed
# score:	footprint
# absolute:	False
# extend:	100
# smooth:	1
# min_limit:	None
# max_limit:	None
# fp_min:	20
# fp_max:	50
# flank_min:	10
# flank_max:	30
# window:	100
# cores:	8
# split:	100
# verbosity:	3


# ----- Outp

In [18]:
# UROPA
import os
import subprocess
from concurrent.futures import ProcessPoolExecutor

# Directory containing peak files
peaks_dir = 'neurogenesis/peaks'
output_dir = 'neurogenesis/uropa_output'
os.makedirs(output_dir, exist_ok=True)

# GTF file path
gtf_file = "/groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/Homo_sapiens.GRCh38.113.chr.gtf"

# Function to run uropa for a single peak file
def run_uropa(peak_file):
    print(f"Running uropa for: {peak_file}")
    command = [
        "uropa",
        "--bed", peak_file,
        "--gtf", gtf_file,
        "--show_attributes", "gene_id", "gene_name",
        "--feature_anchor", "start",
        "--distance", "20000", "10000",
        "--feature", "gene",
        "--outdir", output_dir
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
    print(result.stderr.decode())

# Get all .bed files in the peak directory
peak_file = os.path.join(peaks_dir, "trajectory_ALL.bed")

# Run in parallel
run_uropa(peak_file)

Running uropa for: neurogenesis/peaks/trajectory_ALL.bed
2025-11-05 13:15:08 (3325253) [INFO]	Started UROPA 4.0.3
2025-11-05 13:15:08 (3325253) [INFO]	Working directory: /groups/mb928_gp/mp4486/tobias
2025-11-05 13:15:08 (3325253) [INFO]	Command-line call: /users/mp4486/.local/bin/uropa --bed neurogenesis/peaks/trajectory_ALL.bed --gtf /groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/Homo_sapiens.GRCh38.113.chr.gtf --show_attributes gene_id gene_name --feature_anchor start --distance 20000 10000 --feature gene --outdir neurogenesis/uropa_output
2025-11-05 13:15:08 (3325253) [INFO]	Reading configuration from commandline/input config
2025-11-05 13:15:08 (3325253) [INFO]	Preparing .gtf-file for fast access
2025-11-05 13:15:29 (3325253) [WARNING]	Indexing failed - the GTF is probably unsorted
2025-11-05 13:15:29 (3325253) [WARNING]	Attempting to sort with call: grep -v "^#" neurogenesis/uropa_output/trajectory_ALL_feature_subset.gtf | sort -k1,1 -k4,4n > neurogen

In [19]:
#Cut files
uropa_output_dir = 'neurogenesis/uropa_output'

# Function to run cut header
def run_cut_header(cell_type):
    final_hits = os.path.join(uropa_output_dir, f"{cell_type}_finalhits.txt")
    header_output = os.path.join(uropa_output_dir, f"{cell_type}_annotated_header.txt")
    
    command = f"cut -f 1-6,16-17 {final_hits} | head -n 1 > {header_output}"
    subprocess.run(command, shell=True, check=True)
    print(f"Header saved for {cell_type}")

# Function to run cut body (without header)
def run_cut(cell_type):
    final_hits = os.path.join(uropa_output_dir, f"{cell_type}_finalhits.txt")
    body_output = os.path.join(uropa_output_dir, f"{cell_type}_annotated.bed")
    
    command = f"cut -f 1-6,16-17 {final_hits} | tail -n +2 > {body_output}"
    subprocess.run(command, shell=True, check=True)
    print(f"Annotated BED saved for {cell_type}")

# Parallel execution
run_cut_header('trajectory_ALL')
run_cut('trajectory_ALL')

Header saved for trajectory_ALL
Annotated BED saved for trajectory_ALL


In [20]:
cell_types

['NSC.a', 'NSC.b', 'INP', 'NB', 'ImGC.1', 'ImGC.2', 'GC.1']

In [12]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4" 

In [21]:
# Run bin detect

genome_fasta = "/groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/genome.fa"
motif_file = "/groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/JASPAR2024.jaspar"
uropa_output_dir = 'neurogenesis/uropa_output'

def run_bin_detect(cell_types):
    base = 'neurogenesis/merged_bams/ALL_compare'
    ct0_fp = f"{base}/{cell_types[0]}_footprints/{cell_types[0]}_footprints.bw"
    ct1_fp = f"{base}/{cell_types[1]}_footprints/{cell_types[1]}_footprints.bw"
    ct2_fp = f"{base}/{cell_types[2]}_footprints/{cell_types[2]}_footprints.bw"
    ct3_fp = f"{base}/{cell_types[3]}_footprints/{cell_types[3]}_footprints.bw"
    ct4_fp = f"{base}/{cell_types[4]}_footprints/{cell_types[4]}_footprints.bw"
    ct5_fp = f"{base}/{cell_types[5]}_footprints/{cell_types[5]}_footprints.bw"
    ct6_fp = f"{base}/{cell_types[6]}_footprints/{cell_types[6]}_footprints.bw"

    
    peaks = f"{uropa_output_dir}/trajectory_ALL_annotated.bed"
    header = f"{uropa_output_dir}/trajectory_ALL_annotated_header.txt"
    output_dir = f"neurogenesis/ALL_BINDETECT"
    os.makedirs(output_dir, exist_ok=True)

    cmd = [
        "TOBIAS", "BINDetect",
        "--motifs", motif_file,
        "--signals", ct0_fp, ct1_fp,ct2_fp, ct3_fp,ct4_fp, ct5_fp,ct6_fp,
        "--genome", genome_fasta,
        "--peaks", peaks,
        "--peak_header", header,
        "--outdir", output_dir,
        "--cond_names", f"{cell_types[0]}", f"{cell_types[1]}", f"{cell_types[2]}", f"{cell_types[3]}", f"{cell_types[4]}", f"{cell_types[5]}", f"{cell_types[6]}",
        "--cores", "4"
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"BINDetect ran for ALL")
    except subprocess.CalledProcessError as e:
        print(f"Error running BINDetect ALL")

# Run in parallel
run_bin_detect(cell_types)

# TOBIAS 0.17.1 BINDetect (run started 2025-11-05 13:15:47.670800)
# Working directory: /groups/mb928_gp/mp4486/tobias
# Command line call: TOBIAS BINDetect --motifs /groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/JASPAR2024.jaspar --signals neurogenesis/merged_bams/ALL_compare/NSC.a_footprints/NSC.a_footprints.bw neurogenesis/merged_bams/ALL_compare/NSC.b_footprints/NSC.b_footprints.bw neurogenesis/merged_bams/ALL_compare/INP_footprints/INP_footprints.bw neurogenesis/merged_bams/ALL_compare/NB_footprints/NB_footprints.bw neurogenesis/merged_bams/ALL_compare/ImGC.1_footprints/ImGC.1_footprints.bw neurogenesis/merged_bams/ALL_compare/ImGC.2_footprints/ImGC.2_footprints.bw neurogenesis/merged_bams/ALL_compare/GC.1_footprints/GC.1_footprints.bw --genome /groups/mb928_gp/adr2189/TOBIAS/Tobias_footprinting_non_GN/ATACorrect_input/genome.fa --peaks neurogenesis/uropa_output/trajectory_ALL_annotated.bed --peak_header neurogenesis/uropa_output/trajectory_ALL_annotated_header.txt --o

OpenBLAS warning: precompiled NUM_THREADS exceeded, adding auxiliary array for thread metadata.
To avoid this warning, please rebuild your copy of OpenBLAS with a larger NUM_THREADS setting
or set the environment variable OPENBLAS_NUM_THREADS to 64 or lower


2025-11-05 13:19:11 (3325301) [STATS]	- Threshold estimated at: 1.31091

2025-11-05 13:19:11 (3325301) [INFO]	Calculating background log2 fold-changes between conditions
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.a / NSC.b
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.a / INP
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.a / NB
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.a / ImGC.1
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.a / ImGC.2
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.a / GC.1
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.b / INP
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.b / NB
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.b / ImGC.1
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.b / ImGC.2
2025-11-05 13:19:11 (3325301) [INFO]	- NSC.b / GC.1
2025-11-05 13:19:11 (3325301) [INFO]	- INP / NB
2025-11-05 13:19:11 (3325301) [INFO]	- INP / ImGC.1
2025-11-05 13:19:11 (3325301) [INFO]	- INP / ImGC.2
2025-11-05 13:19:11 (3325301) [INFO]	- INP / GC.1
2025-11-05 13:19:11 (3325301) [INFO]	- NB / ImGC.1
20

/users/mp4486/.local/lib/python3.12/site-packages/tobias/tools/bindetect.py:764: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  info_table.at[names[i], base + "_highlighted"] = False
/users/mp4486/.local/lib/python3.12/site-packages/tobias/tools/bindetect.py:764: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  info_table.at[names[i], base + "_highlighted"] = False
/users/mp4486/.local/lib/python3.12/site-packages/tobias/tools/bindetect.py:764: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

2025-11-05 13:57:02 (3325301) [INFO]	Creating BINDetect plot(s)
2025-11-05 13:57:02 (3325301) [INFO]	- NSC.a / NSC.b (static plot)
68 [0.25495647 0.37488142]
69 [-0.25925166 -0.8335346 ]
30 [ 0.25507171 -0.11753086]
80 [0.43204032 0.98744872]
2025-11-05 13:57:17 (3325301) [INFO]	- NSC.a / NSC.b (interactive plot)
2025-11-05 13:57:17 (3325301) [INFO]	- NSC.a / INP (static plot)
19 [-0.07472483 -0.91331964]
59 [-0.68203987 -0.32608114]
2025-11-05 13:57:32 (3325301) [INFO]	- NSC.a / INP (interactive plot)
2025-11-05 13:57:32 (3325301) [INFO]	- NSC.a / NB (static plot)
2025-11-05 13:57:47 (3325301) [INFO]	- NSC.a / NB (interactive plot)
2025-11-05 13:57:47 (3325301) [INFO]	- NSC.a / ImGC.1 (static plot)
13 [-0.69615611  0.04328604]
14 [ 0.60515605 -0.56290986]
46 [0.7726982  0.58615178]
47 [-0.51934823 -0.09855888]
90 [0.50907788 0.80319537]
91 [-0.17090837 -0.0329486 ]
2025-11-05 13:58:02 (3325301) [INFO]	- NSC.a / ImGC.1 (interactive plot)
2025-11-05 13:58:02 (3325301) [INFO]	- NSC.a / I

In [18]:
!pwd

/groups/mb928_gp/mp4486/tobias
